In [1]:
import pyaudio
import wave
import sys
import os
import ctypes
import struct
import scipy.io.wavfile as wav
import numpy as np
from time import sleep
from collections import Counter
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.io import wavfile

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import cluster
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedKFold
import keras

%matplotlib inline

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def parse_one_file(filepath):
    res = []
    with open(filepath, 'r') as inf:
        for line in inf:
            res.append(map(float, line.split(' ')[:-1]))
    return res


def get_all_data(path='/home/kolegor/Code/VAS/data/wav_files_features/'):
    X, y = [], []
    for folder in os.listdir(path):
        for filename in os.listdir(path + folder):
            filepath = path + folder + '/' + filename
            class_ = int(folder.split('_')[-1])
            features = parse_one_file(filepath)
            X.extend(features)
            y.extend([class_] * len(features))
    assert(len(X) == len(y))
    return X, y

In [3]:
X, y = get_all_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Random Forest

In [16]:
def random_forest(X_train, X_test, y_train, y_test):    
    rf = RandomForestClassifier(n_estimators=300)
    rf.fit(X_train, y_train)
    return rf.score(X_test, y_test)

In [17]:
for i in xrange(10):
    X, y = get_all_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42 + i)
    
    score = random_forest(X_train, X_test, y_train, y_test)
    print score

0.937449879711
0.93825180433
0.927826784282
0.927024859663
0.92141138733
0.928628708901
0.918203688853
0.925421010425
0.925421010425
0.927826784282


### Neural net

In [6]:
def create_nn(input_dim, number_of_classes):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(input_dim, input_dim=input_dim, activation='relu'))
    model.add(keras.layers.Dense(input_dim, activation='relu'))
    model.add(keras.layers.Dense(number_of_classes, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def get_nn_score(nn, X_train, X_test, y_train, y_test):    
    # nn = create_nn(len(X_train[0]), len(set(y_train + y_test)))
    y_train = np.array(map(lambda x: x - 1, y_train)).reshape((-1, 1))
    y_test = np.array(map(lambda x: x - 1, y_test)).reshape((-1, 1))
    
    nn.fit(X_train, y_train, verbose=False)
    return nn.predict_proba(X_test)
    #scores = nn.evaluate(X_test, y_test, verbose=False)
    #return zip(nn.metrics_names, scores)
    # scores = get_nn_score(nn, X_train, X_test, y_train, y_test)


def cross_validation(X, y):
    n_folds = 10
    data, labels = np.array(X), np.array(y)
    skf = StratifiedKFold(labels, n_folds=n_folds, shuffle=True)

    for i, (train, test) in enumerate(skf):
        model = None # Clearing the NN.
        model = create_nn(len(data[0]), len(y))
        # scores = get_nn_score(model, data[train], data[test], labels[train], labels[test])
        scores = get_nn_score(model, data, data, labels, labels)
        print scores

In [43]:
qwe = get_nn_score(create_nn(len(X_train[0]), len(set(y_train + y_test))), X_train, X_test, y_train, y_test)

 800/1247 [==================>...........] - ETA: 0s

In [53]:
for test, real in zip(qwe, y_test):
    asd = zip(test, range(1, len(test) + 1))
    max_ = max(asd, key=lambda x: x[0])
    if max_[1] != real:
        print '\n', real, max_[1]
        print asd[real - 1]
        print max_


1 5
(0.00031666327, 1)
(0.00065628154, 5)

2 4
(0.00094743219, 2)
(0.0065671573, 4)

6 3
(0.055104714, 6)
(0.15716927, 3)

6 3
(0.000610716, 6)
(0.00094257016, 3)

2 6
(0.0053401878, 2)
(0.8361119, 6)

1 7
(0.0036717118, 1)
(0.11686309, 7)

3 1
(0.084046081, 3)
(0.11164274, 1)

3 4
(0.0028063366, 3)
(0.025034422, 4)

1 3
(0.0030604226, 1)
(0.032412548, 3)

4 2
(0.19754107, 4)
(0.49456057, 2)

4 2
(0.63534045, 4)
(0.65208471, 2)

6 4
(0.066274136, 6)
(0.22503877, 4)

4 2
(3.2029118e-05, 4)
(0.00044013423, 2)

4 3
(0.076073237, 4)
(0.086032286, 3)

4 5
(0.00091409363, 4)
(0.0012867337, 5)

3 4
(0.13139945, 3)
(0.15644029, 4)

3 4
(0.0054587647, 3)
(0.21562105, 4)

4 3
(0.07415802, 4)
(0.28339064, 3)

1 5
(0.00094791799, 1)
(0.0016347397, 5)

1 4
(8.8763954e-05, 1)
(0.001541973, 4)

4 3
(0.03251652, 4)
(0.61112583, 3)

6 3
(0.0092486376, 6)
(0.089359164, 3)

2 4
(0.77614701, 2)
(0.96075243, 4)

1 4
(0.00014821901, 1)
(0.00022967592, 4)

1 4
(4.8952548e-05, 1)
(0.0047075837, 4)

2 4
(0.00

### Clustering

In [19]:
def get_score_with_clusters(X_train, X_test, y_train, y_test, data_transformation_function=get_same_data, pca_number=10):
    # preparse data for clustering
    y = y_train + y_test
    X = X_test + X_train
    n_clusters = len(set(y))
    
    X = data_transformation_function(X, pca_number)
    X = np.array(X)
    
    # initialize algos
    bandwidth = cluster.estimate_bandwidth(X, quantile=0.3)
    connectivity = kneighbors_graph(X, n_neighbors=10, include_self=False)
    connectivity = 0.5 * (connectivity + connectivity.T)

    # create clustering estimators
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    k_means = cluster.MiniBatchKMeans(n_clusters=n_clusters)
    ward = cluster.AgglomerativeClustering(n_clusters=n_clusters, linkage='ward', connectivity=connectivity)
    spectral = cluster.SpectralClustering(n_clusters=n_clusters, eigen_solver='arpack', affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=.2)
    affinity_propagation = cluster.AffinityPropagation(damping=.9,preference=-200)
    average_linkage = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock", n_clusters=n_clusters,connectivity=connectivity)
    birch = cluster.Birch(n_clusters=n_clusters)
    
    # initialize iteration variables
    clustering_algorithms = [
        k_means, affinity_propagation, ms, spectral, ward, average_linkage,
        dbscan, birch
    ]
    clustering_names = [
        'MiniBatchKMeans', 'AffinityPropagation', 'MeanShift',
        'SpectralClustering', 'Ward', 'AgglomerativeClustering',
        'DBSCAN', 'Birch'
    ]
    
    results = []
    for cluster_algo, cluster_name in zip(clustering_algorithms, clustering_names):
        print 'Running {0}'.format(cluster_name)
        algo = cluster_algo.fit(X)
        
        if hasattr(cluster_algo, 'labels_'):
            y_pred = cluster_algo.labels_.astype(np.int)
        else:
            y_pred = cluster_algo.predict(X)            
        
        results.append((accuracy_score(y, y_pred), zip(y, y_pred)))  
    return clustering_names, results

In [20]:
cluster_names, cluster_research_results = get_score_with_clusters(X_train, X_test, y_train, y_test)

for name, results in zip(cluster_names, cluster_research_results):
    print name, results[0]

Running MiniBatchKMeans
Running AffinityPropagation
Running MeanShift
Running SpectralClustering


/usr/local/lib/python2.7/dist-packages/sklearn/manifold/spectral_embedding_.py:229: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Running Ward


/usr/local/lib/python2.7/dist-packages/sklearn/cluster/hierarchical.py:193: UserWarning: the number of connected components of the connectivity matrix is 2 > 1. Completing it to avoid stopping the tree early.
  connectivity, n_components = _fix_connectivity(X, connectivity)


Running AgglomerativeClustering


/usr/local/lib/python2.7/dist-packages/sklearn/cluster/hierarchical.py:418: UserWarning: the number of connected components of the connectivity matrix is 2 > 1. Completing it to avoid stopping the tree early.
  connectivity, n_components = _fix_connectivity(X, connectivity)


Running DBSCAN
Running Birch
MiniBatchKMeans 0.0966375430236
AffinityPropagation 0.0
MeanShift 0.0
SpectralClustering 0.0831347630394
Ward 0.0746624305004
AgglomerativeClustering 0.00132380195923
DBSCAN 0.0
Birch 0.0979613449828


## Testing models in production

In [4]:
import sys
# sys.path.append('/home/kolegor/Code/VAS/system/source/python_feature_exctractors/')
import py_model
reload(py_model)
reload(py_model)

<module 'py_model' from '/home/kolegor/Code/VAS/system/source/python_feature_exctractors/py_model.pyc'>

In [7]:
qwe = py_model.NeuralNetModel('/home/kolegor/', input_dim=5, number_of_classes=2)

In [11]:
qwe.train_and_save(1, 2)

Exception: BaseModel::train_and_save(...). Not implemented yet. Need to be reloaded